# Data preprocessing

* Resample all data into the same spatial resolution (2x2 degrees).
  
* Combine all types of emissions into one feature.
  
* Unify the sampling frequency of all data into every 3 hours.

In [24]:
import numpy as np
import pandas as pd
import dask.array as da
import xarray as xr
import rasterio
from rasterio.warp import Resampling
import matplotlib.pyplot as plt

In [25]:
from resample import resampling, yearly_resample, days_in_month, interpolate_daily

In [26]:
# Parameters for resampling
# data path
emission_data_path = '../data/'
# target resolution or target shape
target_resolution = 2
target_shape = (90,180) # 180/2, 360/2

## Emission

* 3 emissions are included: Anthropogenic, Wetland, Fire

* Unify the unit of 3 emissions: convert the unit of Wetland emission from `kg m-2 mon-1` to `kg m-2 s-1`.
  
* Resample each emission separately and then sum them together.

* Extand the sampling frequency of emissions to every 3 hours.

In [36]:
# Anthropogenic Emission: (1800,3600)->(90,180) : 0.1->2
ant_emission = xr.open_mfdataset(emission_data_path + 'anthropogenic/*.nc')
print("sum before resample: ", ant_emission['sum'].values.sum())
ant_emission['sum'] = resampling(ant_emission['sum'], target_resolution, target_shape, target_crs = "EPSG:4326", resampler=Resampling.bilinear)
print("sum after resample: ", ant_emission['sum'].values.sum()*(2/0.1)**2)
ant_emission.to_netcdf(emission_data_path + "resample/emissions/anthro.nc")

sum before resample:  0.0055805813
sum after resample:  0.005580697688856162


In [52]:
# Wetland Emission: (360,720)->(90,180) : 0.5->2
wetland_emission = xr.open_mfdataset(emission_data_path + r'wetland/*.nc',)
# convert unit from kg m-2 mon-1 to kg m-2 s-1
time_dim = wetland_emission['time']
seconds_in_month = days_in_month(time_dim) * 24 * 60 * 60
seconds_in_month_da = xr.DataArray(seconds_in_month, dims=['time'], coords={'time': time_dim})
wetland_emission['ch4_wl'] = wetland_emission['ch4_wl'] / seconds_in_month_da
wetland_emission['ch4_wl'].attrs['units'] = 'kg CH4 m-2 s-1'
print("sum before resample: ", wetland_emission['ch4_wl'].values.sum())
wetland_emission = resampling(wetland_emission['ch4_wl'], target_resolution,target_shape , target_crs = "EPSG:4326", resampler=Resampling.bilinear)
print("sum after resample: ", wetland_emission.values.sum()*(2/0.5)**2)
wetland_emission = wetland_emission.to_dataset()
wetland_emission.to_netcdf(emission_data_path + "resample/emissions/wetland.nc")

sum before resample:  9.519453348091839e-05
sum after resample:  9.519605213002704e-05


In [50]:
# Fire Emission
fire_emission = xr.open_dataset(emission_data_path + 'fire/CAMS_fire_0.1x0.1_2015-2018.nc', chunks={'time':1})
fire_emission = yearly_resample(fire_emission['ch4fire'], target_resolution,target_shape , target_crs = "EPSG:4326", resampler=Resampling.bilinear)
fire_emission.to_netcdf(emission_data_path + "resample/emissions/fire.nc")

In [54]:
print("sum after resample: ", fire_emission['ch4fire'].values.sum()*(2/0.1)**2)

sum after resample:  0.008235043108086653


### Calculate sum of 3 emissions

In [56]:
# interpolate data to daily
wetland_daily = interpolate_daily(wetland_emission['ch4_wl'])
ant_daily = interpolate_daily(ant_emission['sum'])
fire_daily = fire_emission['ch4fire'].reindex_like(wetland_daily, method='nearest')
ant_daily = ant_daily.reindex_like(wetland_daily, method='nearest')
total_emission = wetland_daily + ant_daily + fire_daily
total_emission = xr.Dataset({'sum':total_emission})
total_emission['sum'].attrs['units'] = 'kg m-2 s-1'
total_emission['sum'].attrs['long_name'] = 'Total emission for CH4'
total_emission.to_netcdf(emission_data_path + "resample/emissions_daily.nc")

In [57]:
# extand data to every 3 hours
all_times = pd.date_range(start=total_emission['time'].values.min(), end=total_emission['time'].values.max() + pd.Timedelta(days=1), freq='3H')[:-1]
emission_hours = total_emission.reindex(time=all_times, method='nearest').ffill('time')
emission_hours.to_netcdf(emission_data_path + "resample/emissions.nc")

## Methane Concentration

In [4]:
methane = xr.open_dataset(emission_data_path + "total_column_methane.nc", chunks={'time':1})
methane = yearly_resample(methane['tc_ch4'], target_resolution,target_shape , target_crs = "EPSG:4326", resampler=Resampling.bilinear)
methane.to_netcdf(emission_data_path + "resample/total_column_methane.nc")

## Meteorological data

In [4]:
meteorology = xr.open_dataset(emission_data_path + "meteorology/meteorology.nc", chunks={'time':1})
vars = list(meteorology.data_vars)
"""
u10: 10m u-component of wind
v10: 10m v-component of wind
t2m: 2m temperature
z: surface geopotential 地面重力势能
tcwv: Total column water vapour
"""
print(vars)

['u10', 'v10', 't2m', 'z', 'tcwv']


In [7]:
meteo_resample = {}
for var in vars:
    data = yearly_resample(meteorology[var], target_resolution,target_shape , target_crs = "EPSG:4326", resampler=Resampling.bilinear)
    meteo_resample[var] = data[var]
meteo_resample = xr.Dataset(meteo_resample)
meteo_resample.to_netcdf(emission_data_path + f"resample/meteorology.nc")